In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gensim
print(os.listdir("../input/embeddings/GoogleNews-vectors-negative300/"))
path="../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin"
embeddings=gensim.models.KeyedVectors.load_word2vec_format(path,binary=True)

['GoogleNews-vectors-negative300.bin']


In [2]:
len(embeddings['amazon'])

300

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([embeddings['camera'],embeddings['photo']])

array([[1.0000002 , 0.46263507],
       [0.46263507, 0.99999994]], dtype=float32)

In [4]:
# embeddings.most_similar('hyundai',topn=10)
# embeddings.doesnt_match(['rahul','sonia','gandhi','sachin'])
# embeddings.doesnt_match(['football','cricket','grape'])
embeddings.most_similar(positive=['king','woman'],negative=['man'],topn=1)

[('queen', 0.7118192911148071)]

In [5]:
url='https://raw.githubusercontent.com/skathirmani/datasets/master/imdb_sentiment.csv'
imdb=pd.read_csv(url)
imdb.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


### Since sparce matrix is large dimension we need to create a vector representation of documents instead of words

In [6]:
import nltk

In [7]:
stopwords=nltk.corpus.stopwords.words('english')
docs_vector=pd.DataFrame()
for doc in imdb['review'].str.lower().str.replace('[^a-z ]',''):
    words=nltk.word_tokenize(doc)
    words=[word for word in words if word not in stopwords]
    temp=pd.DataFrame()
    for word in words:
        try:
            word_vec=pd.Series(embeddings[word])
            temp=temp.append(word_vec,ignore_index=True)
        except:
            pass
    temp_avg=temp.mean()
    docs_vector=docs_vector.append(temp_avg,ignore_index=True)

In [8]:
docs_vector.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.170980,0.169840,-0.111511,0.045278,-0.063273,0.021952,-0.062159,-0.018341,0.048421,0.052041,0.042328,-0.050680,-0.047974,0.056483,-0.167155,-0.086711,-0.054972,0.155884,-0.040588,-0.074300,0.064209,0.113986,-0.016048,-0.026428,0.111348,-0.032374,-0.006551,0.216349,0.228109,-0.046229,-0.020376,-0.016744,-0.031245,-0.020854,0.013041,0.046427,0.058441,-0.105367,0.010366,0.130208,...,-0.070292,-0.029043,-0.032211,-0.038025,-0.006877,0.166992,-0.087087,-0.043701,-0.091003,-0.034032,-0.142291,0.124573,0.062215,0.032410,0.004954,-0.004415,-0.058146,-0.005880,-0.045044,0.063070,-0.018799,-0.128042,0.064128,0.104614,-0.016479,0.079224,-0.117269,0.052139,0.077515,0.054382,-0.073405,0.086751,-0.090210,-0.075134,-0.072550,0.062012,0.072103,-0.005147,0.044312,0.040568
1,0.143494,-0.017984,-0.009964,0.106308,-0.052292,-0.075745,-0.015942,-0.037323,0.114197,0.029335,-0.003784,-0.073822,0.036469,0.065918,-0.102180,0.042419,0.084106,0.104202,0.008781,0.016678,-0.073910,0.029068,0.069580,-0.020123,0.104767,-0.059865,-0.115555,0.106262,0.039726,-0.030182,-0.165405,-0.049149,-0.125187,0.011609,0.058807,-0.049637,0.005493,0.000763,-0.068726,0.156372,...,-0.077637,-0.048020,-0.022751,0.078697,-0.027492,0.164474,0.057610,-0.035126,-0.076599,0.050873,-0.014007,0.020390,0.030361,-0.075378,0.131317,-0.044189,-0.149719,-0.022987,-0.029526,-0.013031,0.008360,-0.029116,-0.049454,0.133499,0.112343,-0.035961,-0.119406,0.032257,0.043850,0.023603,-0.080658,0.051147,-0.152344,-0.048691,-0.065536,-0.029953,0.032959,-0.053650,0.021364,-0.096176
2,0.129862,0.047370,-0.014943,0.006382,-0.087040,-0.003391,0.066913,0.016049,0.062988,0.037282,0.009698,-0.101617,-0.029222,-0.015354,-0.146093,0.037862,0.046577,0.098209,-0.004571,-0.089184,-0.022402,0.030077,0.047251,-0.037550,0.082318,-0.068688,-0.107069,0.109409,0.069038,0.001902,-0.037581,0.007484,0.030528,-0.004909,0.044056,0.043783,0.060629,0.001321,0.029723,0.019997,...,-0.076128,0.024412,-0.042903,-0.019151,0.050251,0.174241,-0.074519,-0.123002,-0.095108,-0.047004,0.004748,0.090578,0.029965,0.017436,0.086934,0.000882,-0.064568,-0.064304,-0.017856,0.098301,-0.003079,-0.062395,0.054640,0.028648,-0.042235,0.044403,-0.126282,-0.017722,0.008898,-0.076619,-0.115946,0.073004,-0.129066,0.035997,-0.082231,-0.017315,0.063343,-0.051036,0.023688,0.027690
3,0.070648,-0.059143,-0.003357,0.084737,-0.092316,0.107788,0.156372,-0.170471,0.006287,-0.007477,0.011261,-0.155518,-0.103271,0.043068,-0.077423,0.129700,0.086182,0.225586,0.098694,-0.126465,-0.036331,0.118652,0.010071,-0.095245,0.036072,0.069336,-0.080078,0.011114,0.031620,0.006650,-0.023468,0.107666,-0.121738,-0.051147,0.073364,-0.055099,0.157959,-0.013733,0.059799,0.034119,...,-0.050415,-0.023674,-0.136963,-0.063477,-0.105469,0.217285,-0.020557,-0.032013,-0.193604,-0.080750,0.049652,0.102173,0.208252,0.035278,0.086731,-0.142639,-0.062012,-0.071838,-0.023804,-0.102173,-0.015747,-0.009747,0.089417,0.077164,0.084160,0.047089,0.008484,-0.093872,-0.068604,0.142441,-0.052734,0.080322,-0.023529,0.030060,-0.011902,-0.059326,0.144897,-0.071167,0.074524,-0.052612
4,0.120809,0.007568,0.040127,0.048381,-0.056264,0.004886,0.024143,-0.053096,0.066806,-0.012214,0.037859,-0.098823,-0.032728,0.001539,-0.148058,0.081838,-0.031436,0.093245,-0.020969,-0.118635,0.024455,0.153320,-0.051242,-0.025713,0.024929,-0.111016,-0.051229,0.081462,0.043762,-0.051346,-0.055230,0.009820,-0.001556,0.023844,0.085300,-0.014167,0.042087,0.006429,0.050415,0.127075,...,-0.090875,0.057387,0.024984,0.048028,0.034254,0.131344,-0.027271,-0.049398,-0.088830,0.061483,-0.001966,0.093648,0.094103,0.100993,0.034740,0.011668,-0.015808,-0.157376,-0.056332,0.013950,-0.030368,-0.031494,-0.023329,0.084703,0.020335,0.044976,-0.160895,0.036072,0.090658,0

In [9]:
docs_vector.isnull().sum().sum()

600

In [10]:
docs_vector['sentiment']=imdb['sentiment']
docs_vector=docs_vector.dropna()

In [11]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
train,test=train_test_split(docs_vector,test_size=0.3,random_state=100)
train_x=train.drop('sentiment',axis=1)
test_x=test.drop('sentiment',axis=1)
train_y=train['sentiment']
test_y=test['sentiment']

In [13]:
model_ab=AdaBoostClassifier(n_estimators=300,random_state=100)
model_ab.fit(train_x,train_y)
pred=model_ab.predict(test_x)
accuracy_score(test_y,pred)

0.8169642857142857

In [14]:
model_rf=RandomForestClassifier(n_estimators=300,random_state=100)
model_rf.fit(train_x,train_y)
pred=model_rf.predict(test_x)
accuracy_score(test_y,pred)

0.8080357142857143

In [ ]:
model_xg=XGBClassifier(learning_rate=0.43)
model_xg.fit(train_x,train_y)
pred=model_xg.predict(test_x)
accuracy_score(test_y,pred)

In [ ]:
model_gb=GradientBoostingClassifier(n_estimators=300,random_state=100)
model_gb.fit(train_x,train_y)
pred=model_gb.predict(test_x)
accuracy_score(test_y,pred)